# Copyright : fast.ai - Jeremy Howard & Sylvain Gugger - 2020 (GPLv3)

Cellules de code et plan du notebook adaptées du livre :

Deep Learning for Coders with fastai & PyTorch de Jeremy Howard et Sylvain Gugger.

The code in the original notebooks (and thus the code in this notebook) is covered by the GPL v3 license; see the LICENSE file for details.

# NLP Deep Dive: RNNs

Nous allons entrainer un **language model** : un modèle capable de deviner le mot suivant dans un texte à partir des mots qui le précèdent. 

Ce type de tâche est appelé **self-supervised learning** : nous n'avons pas besoin de créer explicitement des labels pour entrainer ce type de modèle, il suffit de le nourrir de beaucoup, beaucoup de textes. Pour cette tâche, nous avons un processus pour obtenir automatiquement les labels à prédire à partir des données brutes.

Cette tâche n'est pas triviale : pour deviner correctement le mot suivant dans une phrase, le modèle devra développer une compréhension assez approfondie de la langue. 

Le **self-supervised learning** peut également être utilisé dans d'autres domaines, c'est un sujet de recherche très actif en ce moment, par exemple en vision : [Prototypical Contrastive Learning](https://blog.einstein.ai/prototypical-contrastive-learning-pushing-the-frontiers-of-unsupervised-learning/). 

Un modèle entrainé par **self-supervised learning** n'est en général pas utilisé directement : il sert de point de départ pour réaliser un fine-tuning en mode supervisé sur une beaucoup plus petite quantité de données.

Nous allons travailler sur le jeu de données [IMDb sentiment analysis](https://www.imdb.com/title/tt0107144/reviews?ref_=tt_ov_rt) :
- 50.000 critiques de films sans labels positifs ou négatifs
- 25 000 critiques avec des labels dans le jeu d'entrainement
- 25 000 critiques avec des labels dans le jeu de validation
- => 100 000 critiques de films au total

Nous pouvons utiliser toutes ces critiques pour affiner un language model pré-entrainé sur les articles de Wikipédia : cela permettra d'obtenir un language model particulièrement efficace pour prédire le prochain mot d'une critique de film.

Cette approche appelée [Universal Language Model Fine-tuning (ULMFit)](https://arxiv.org/abs/1801.06146) a été inventée lors du cours fastai 2017-2018. C'était le premier exemple de transfert d'apprentissage sur du texte. Des benchmarks sont régulièrement réalisés avec des méthodes plus modernes et beaucoup plus lourdes en termes de calcul : cette méthode reste un très bon compromis coût/performance pour la classification de textes.

Cette publication a montré que l'étape supplémentaire de fine tuning du language model, avant de transférer l'apprentissage à une tâche de classification, permettait d'obtenir des prévisions nettement meilleures.

En utilisant cette approche, nous avons trois étapes pour le transfert de l'apprentissage en NLP (Natural Language Processing) :
1. Language model - self-supervised - entrainement initial : Wikitext 103
2. Language mode - self-supervised - fine tuning : IMDb
3. Classifier - supervisé - fine tuning : IMDb

## Text Preprocessing

Nous avons déjà vu comment les variables de type catégorie (énumération - liste de niveaux finie) peuvent être utilisées en entrée d'un réseau de neurones.

L'approche que nous avons adoptée était la suivante :
1. Faire une liste de toutes les valeurs possibles de cette catégorie (nous appellerons cette liste le vocabulaire).
2. Remplacer chaque valeur par son index dans le vocabulaire.
3. Créer une matrice d'embeddings contenant une représentation de chaque valeur (c'est-à-dire pour chaque élément du vocabulaire) sous forme d'une liste de nombres .
4. Utiliser cette matrice d'embeddings comme première couche d'un réseau de neurones. 

Les mots de vocabulaire d'une langue donnée peuvent être traités comme les différents niveaux d'une variable de type catégorie : on va retenir une liste finie de mots et **apprendre une représentation numérique sous forme d'embedding pour chacun des mots du vocabulaire retenu**.

Ce qui est nouveau, c'est l'idée d'une **séquence**. D'abord, nous concaténons tous les documents de notre jeu de données en une longue chaîne et nous la divisons en mots, ce qui nous donne une très longue liste de mots (ou "tokens"). La donnée d'entrée de notre modèle sera la séquence de mots commençant par le premier mot de notre très longue liste et se terminant par l'avant-dernier, et notre variable à prédire sera la séquence de mots commençant par le deuxième mot et se terminant par le dernier.

Notre vocabulaire sera constitué d'un mélange de mots communs qui sont déjà dans le vocabulaire de notre modèle pré-entrainé sur Wikipedia et de nouveaux mots spécifiques à notre corpus (termes cinématographiques ou noms d'acteurs, par exemple). Notre matrice d'embeddings sera construite en conséquence : pour les mots qui sont dans le vocabulaire de notre modèle pré-entrainé, nous prendrons la ligne correspondante dans la matrice d'embedding du modèle pré-entrainé ; mais pour les nouveaux mots, nous n'aurons rien, donc nous initialiserons juste la ligne correspondante avec un vecteur aléatoire.

Chacune des étapes nécessaires à la création d'un language model est associée à du jargon issu du monde du traitement du langage naturel, et des classes de fastai et PyTorch sont disponibles pour aider. Les étapes sont les suivantes :

- Tokenization : Convertir le texte en une liste de mots (ou de caractères, ou de sous-chaînes de caractères, selon la granularité de votre modèle)
- Numericalization : Faire une liste de tous les mots uniques qui apparaissent (le vocabulaire), et convertir chaque mot en un nombre, en recherchant son index dans le vocabulaire
- Création d'un DataLoader pour le language model : fastai fournit une classe LMDataLoader qui gère automatiquement la création d'une variable à prédire qui est décalée de la variable d'entrée d'un token. Elle gère également certains détails importants, tels que la manière de mélanger les données d'apprentissage de manière à ce que les variables dépendantes et indépendantes conservent leur structure
- Création d'un laguage model : Nous avons besoin d'un type de modèle spécial qui fait quelque chose que nous n'avons jamais vu auparavant : il gère des listes de saisie qui peuvent être arbitrairement grandes ou petites. Il existe plusieurs façons de le faire ; dans ce chapitre, nous utiliserons un réseau neuronal récurrent (RNN).

### Tokenization

Lorsque nous avons parlé de "convertir le texte en une liste de mots", nous avons omis beaucoup de détails. Par exemple, que faisons-nous de la ponctuation ? Comment traiter un mot comme "don't" ? S'agit-il d'un seul mot ou de deux ? Qu'en est-il des longs mots médicaux ou chimiques ? Doivent-ils être séparés en parties distinctes ? Et les mots avec un trait d'union ? Qu'en est-il des langues comme l'allemand et le polonais, où nous pouvons créer des mots vraiment longs à partir de nombreux morceaux ? Qu'en est-il des langues comme le japonais et le chinois qui n'utilisent pas du tout d'espaces et qui n'ont pas vraiment une idée bien définie du mot ?

Comme il n'y a pas de réponse unique à ces questions, il n'y a pas d'approche unique de la tokenisaton. Il existe trois approches principales :

- Word-based : Découper le texte selon les espaces, et appliquer des règles spécifiques à la langue pour essayer de séparer des parties qui ont un sens propre même lorsqu'il n'y a pas d'espaces (comme transformer "don't" en "do not"). En général, les signes de ponctuation sont également divisés en tokens séparés.
- Subword-based : Découper les mots en plus petits morceaux, en vous basant sur les sous-chaînes les plus courantes. Par exemple, "préentrainement" peut être découpé en "pré|entrain|ement".
- Character-based : Découper une phrase en ses différents caractères.

### Word Tokenization with fastai

Plutôt que de fournir son propre tokenizer, fastai fournit une interface cohérente avec des tokenizers fournis par des bibliothèques externes spécialisées. La tokenisation est un domaine de recherche actif, et des tokenizers nouveaux et améliorés sortent en permanence, de sorte que les paramètres par défaut utilisés par fastai changent également. Cependant, l'API et les options ne devraient pas trop changer, car fastai essaie de maintenir une API cohérente même si la technologie sous-jacente change.

In [ ]:
from IPython.display import display,HTML

In [ ]:
from fastai2.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read(); txt[:75]

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

In [ ]:
defaults.text_proc_rules

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

### Subword Tokenization

In [ ]:
txts = L(o.open().read() for o in files[:2000])

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
subword(1000)

In [ ]:
subword(200)

In [ ]:
subword(10000)

### Numericalization with fastai

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

In [ ]:
nums = num(toks)[:20]; nums

In [ ]:
' '.join(num.vocab[o] for o in nums)

### Putting Our Texts into Batches for a Language Model

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
x,y = first(dl)
x.shape,y.shape

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

## Training a Text Classifier

### Language Model Using DataBlock

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

### Fine-Tuning the Language Model

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

### Saving and Loading Models

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

### Text Generation

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

### Creating the Classifier DataLoaders

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

## Disinformation and Language Models

## Conclusion